In [1]:
# Basic modules
import numpy as np
import ir_datasets
# The model modules
from libs.lexical_store import LexicalStore
from libs.vector_store import VectorStore
from libs.retriever import RetrieverQA
from libs.basic_document_loader import BasicIndexer

In [2]:
# Preparing the dataset
# Loading the dataset
dataset = ir_datasets.load("cranfield")
# Preparing the docs and queries
prep_funct = lambda x: dict(doc_id=x[0],title=x[1],text=x[2],author=x[3],bib=x[4])
docs = [doc for doc in map(prep_funct,dataset.docs_iter()) if len(doc['text'])]
queries = {query[0]:query[1] for query in dataset.queries_iter()}


In [3]:
# Preparing the stores
vector = VectorStore()
lexical = LexicalStore()
# Preparing the Indexer
index = BasicIndexer()

In [4]:
# Adding the documents
index.add_documents(docs)
# Preparing the retrieval
doc_text = [doc["text"].strip() for doc in docs ]
vector.add(doc_text)
lexical.add(doc_text)


BM25S Create Vocab:   0%|          | 0/1398 [00:00<?, ?it/s]

BM25S Convert tokens to indices:   0%|          | 0/1398 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/1398 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/1398 [00:00<?, ?it/s]

In [5]:
vector._index.ntotal

1398

In [6]:
# Preparing the retrieval
retriever = RetrieverQA(documents_=index, stores=[vector])

In [7]:
# Iter for al queries
queries_info = {}
for query in dataset.qrels_iter():
    query_id = query[0]
    if query[2] >=3:
        exist_info = queries_info.get(query_id,set())
        exist_info.add(query[1])
        queries_info[query_id] = exist_info


In [ ]:
def get_precistion_recall():
    precision = []
    recall = []
    for query_id, relevants_docs in queries_info.items():
        # The text of the query
        if query_id not in queries:
            continue
    
        query = queries[query_id]
        # Get the documents
        retrieved_docs = set([doc['doc_id'] for doc in retriever.search(query,k=25)])
        # relevant and retrieved
        rr = relevants_docs.intersection(retrieved_docs)
        # Calculating the precision
        current_precision = len(rr)/len(retrieved_docs)
        precision.append(current_precision)
        # Calculating the recall
        current_recall = len(rr)/len(relevants_docs)
        recall.append(current_recall)
    
    precision = np.array(precision)
    recall = np.array(recall)
    
    return precision,recall
    

In [9]:
precision,recall = get_precistion_recall()
print(precision.min(),np.median(precision),precision.mean(),precision.max())
print(recall.min(),np.median(recall),recall.mean(),recall.max())

0.0 0.0 0.009925925925925927 0.28
0.0 0.0 0.05116932208435476 0.6666666666666666


In [ ]:
for index,query in queries.items():
    retrieved = set(map(lambda x: int(x['doc_id']),retriever.search(query)))
    valid =set(map(lambda x:int(x),queries_info[index]))
    print(len(retrieved),len(valid.intersection(retrieved)),len(valid))

25 8 21
25 3 17
25 0 2
25 0 6


KeyError: '9'